<img src="../Img/Python_SQL_PI.png" style="width:100%">


<p style="font-size:22px;text-align:center">Лекция 2. Технология работы с базами данных с помощью Python <br><br> на примере библиотеки <i>sqlite3</i></p> 

24 февраля 2022 года<br>
Поток: ПИ20-1, ПИ20-2, ПИ20-3<br>

28 февраля 2022 года<br>
Поток: ПИ20-4, ПИ20-5, ПИ20-6<br>


<a id = Home></a>

<p style="font-size:18px;text-align:left">Учебные вопросы</p>

1. [Библиотеки для работы с реляционными базами данных в Python](#1)
2. [Создание базы данных SQLite из программы Python.](#2)
3. [Создание таблиц в базе данных SQLite с помощью Python.](#3)
4. [Вставка данных в базу данных SQLite с помощью Python.](#4)
5. [Обновление данных в базе данных SQLite с помощью Python.](#5)
6. [Удаление данных в базе данных SQLite с помощью Python.](#6)
7. [Запросы к данным базы данных SQLite с помощью Python.](#7)

<a id=1></a> [К оглавлению](#Home)

### 1. Работа с базами данных в Python

Популярными инструментами для работы с базами данных в Python являются библиотеки:

sqlite3: https://docs.python.org/3/library/sqlite3.html ; https://pypi.python.org/pypi/pysqlite <br>
APSW: https://rogerbinns.github.io/apsw/ <br>
SQLAlchemy: https://www.sqlalchemy.org/

    
На этом занятии изучим работу с базами данных с помощью библиотеки *sqlite3*.

<a id=2></a>[К оглавлению](#Home)

### 2. Создание базы данных SQLite из программы Python

Библиотека *sqlite3* позволяет создавать и использовать базы данных в формате СУБД SQLite. Официальный сайт этой СУБД: www.sqlite.org

Покажем, как создать новую базу данных SQLite из программы Python с помощью библиотеки *sqlite3*.

Чтобы создать новую базу данных SQLite достаточно создать подключение к ней и файл базы данных будет создан автоматически. Для этого, сначала нам нужно создать объект Connection с помощью функции `connect()`. Например, следующая программа Python создает новый файл базы данных *projects.sqlite* в текущей папке. Если требуется создать файл базы данных в конкретной папке, необходимо в месте с именем файла указать путь к ней, например: ```"D:/Data/pythonsqlite.db"```.

In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

In [3]:
if __name__ == '__main__':
    create_connection("projects.sqlite") # Создание файла БД в текущей папке

2.6.0


Во-первых, мы определяем функцию с именем `create_connection()`, которая подключается к базе данных SQLite, указанной в файле базы данных `db_file`. Внутри функции мы вызываем функцию `connect()` модуля sqlite3.

Функция `connect()` открывает соединение с базой данных SQLite. Он возвращает объект `Connection`, представляющий базу данных. Используя объект `Connection`, мы выполняем различные операции с базой данных.

В случае возникновения ошибки мы перехватываем ее в блоке `try except` и отображаем сообщение об ошибке. Если все в порядке, мы показываем версию базы данных SQLite.

Хорошей практикой является завершение соединения с базой данных, если она больше не используется.

Во-вторых, мы передаем путь к файлу базы данных функции `create_connection()` для создания базы данных.

Существует возможность создания файла БД в оперативной памяти компьютера. Для этого необходимо использовать параметр `:memory:` функции `Connect()` вместо имени файла.

```
create_connection(":memory:")
```

<a id=3></a> [К оглавлению](#Home)

### 3. Создание таблиц базы данных SQLite с помощью Python

Покажем, как создавать таблицы в базе данных SQLite с помощью Python.

Чтобы создать новую таблицу в базе данных SQLite, выполним следующие действия:

1. Напишем выражения, обеспечивающие создание таблиц на языке SQL.
2. Создадим объект *Connection*, используя функцию `connect()` *sqlite3*.
3. Создадим объект *Cursor*, вызвав метод `cursor()` объекта *Connection*.
4. Передадим SQL-выражение методу `execute()` объекта *Cursor* и выполним этот метод.

Создадим две таблицы: "проект" и "задача", как показано на следующей диаграмме:

<img src="../Img/Python-SQLite-Sample-Database.jpg" style="width:50%">

Для создания такой БД потребуется следующий код на языке SQL:

```
-- project table
CREATE TABLE IF NOT EXISTS project (
	id integer PRIMARY KEY,
	name text NOT NULL,
	begin_date text,
	end_date text
);

-- task table
CREATE TABLE IF NOT EXISTS task (
	id integer PRIMARY KEY,
	name text NOT NULL,
	priority integer,
	project_id integer NOT NULL,
	status_id integer NOT NULL,
	begin_date text NOT NULL,
	end_date text NOT NULL,
	FOREIGN KEY (project_id) REFERENCES projects (id)
);
```

Создадим эти таблицы с помощью Python, следуя шагам вышеописанного алгоритма.

In [4]:
create_table_project = """
CREATE TABLE IF NOT EXISTS project (
    id integer PRIMARY KEY,
    name text NOT NULL,
    begin_date text,
    end_date text
); """

create_table_task = """
CREATE TABLE IF NOT EXISTS task (
    id integer PRIMARY KEY,
    name text NOT NULL,
    priority integer,
    project_id integer NOT NULL,
    status_id integer NOT NULL,
    begin_date text NOT NULL,
    end_date text NOT NULL,
    FOREIGN KEY (project_id) REFERENCES projects (id)
);
"""

Напишем две функции - одну для соединения с БД, вторую - для создания в базе данных *projects.sqlite* таблиц проектов и заданий с помощью SQL-выражений CREATE TABLE.

In [5]:
def create_connection(db_file):
    """
    :return: Объект соединения или None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

def create_table(conn, sql_statement):
    try:
        crs = conn.cursor()
        crs.execute(sql_statement)
    except Error as e:
        print(e)

Создадим главную функцию, объединяющую функциональность по созданию таблиц.

In [6]:
def main(database):
    conn = create_connection(database)
    if conn is not None:
        create_table(conn, create_table_project)
        create_table(conn, create_table_task)
    else:
        print("Ошибка! Не могу создать соединение с БД")

И, наконец, чтобы завершить код по созданию таблиц, вызовем главную функцию и передадим ей единственный параметр - имя базы данных.

In [7]:
if __name__ == '__main__':
    main("projects.sqlite")

Чтобы убедиться, что таблицы созданы, откроем базу данных *projects.sqlite* в командном окне *sqlite3* и выполним команду `.tables`.

<img src="../Img/sqlite3_command_line_1.png" style="width:55%">

<a id=4></a> [К оглавлению](#Home)

### 4. Вставка данных в базу данных SQLite с помощью Python

Изучим этапы вставки данных в таблицы базы данных SQLite с помощью Python. Выполним следующие действия:

1. Подключимся к базе данных SQLite, создав объект Connection.
2. Создадим объект Cursor, вызвав метод курсора объекта Connection.
3. Выполним SQL-выражение с оператором INSERT. Чтобы передать аргументы оператору INSERT, используем вопросительный знак (?) в качестве заполнителя для каждого аргумента.

Создадим проект и несколько заданий.

In [8]:
# Функция для создания соединения с базой данных
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

# Функция для добавления проекта
# Создаёт проект, возвращает идентификатор проекта
def create_project(conn, project_values):
    sql = ''' 
    INSERT INTO project (name, begin_date, end_date) VALUES (?, ?, ?)
    '''
    cur = conn.cursor()
    cur.execute(sql, project_values)
    conn.commit()
    return cur.lastrowid

# Функция для добавления заданий проекта
def create_task(conn, task_values):
    """
    Создание задания
    """
    sql = ''' 
    INSERT INTO task(name, priority, project_id, status_id, begin_date, end_date)
        VALUES(?, ?, ?, ?, ?, ?)
        '''
    cur = conn.cursor()
    cur.execute(sql, task_values)
    conn.commit()
    return cur.lastrowid

# Основная функция
def main():
    database = "projects.sqlite"

    # Создание соединения с базой данных
    conn = create_connection(database)
    with conn:
        
        # Создание нового проекта и заданий
        project_1 = ('Cool App with SQLite & Python', '2022-02-10', '2022-02-28');
        project_id = create_project(conn, project_1)
        
        task_1 = ('Analyze the requirements of the app', 1, project_id, 1, '2022-02-10', '2022-02-15')
        task_2 = ('Confirm with user about the top requirements', 1, project_id, 1, '2022-02-15', '2022-02-28')
        task_3 = ('Create documentation', 2, project_id, 1, '2022-02-28', '2022-03-07')
        create_task(conn, task_1)
        create_task(conn, task_2)
        create_task(conn, task_3)
        
        # Создание еще одного проекта
        project_2 = ('Cool App with SQLAlchemy', '2022-03-01', '2022-03-18')
        project_id = create_project(conn, project_2)

    conn.close()

In [9]:
if __name__ == '__main__':
    main()

Подключимся к базе данных *projects.sqlite* с помощью интерфейса командной строки и проверим вставку записей проекта и заданий, выполнив SQL-запросы:

```
.open projects.sqlite
.mode column
select * from project;
select * from task;
```

<img src="../Img/sqlite3_command_line_2.png" style="width:100%">

<a id=5></a> [К оглавлению](#Home)

### 5. Обновление данных

В этом разделе научимся с помощью Python обновлять данные в созданной ранее базе данных. Выполним следующие шаги алгоритма обновления данных.

1. Подключимся к базе данных SQLite, создав объект *Connection*.
2. Создадим объект *Cursor*, вызвав метод `cursor()` объекта *Connection*.
3. Выполним команду `UPDATE` с помощью вызова метода `execute()` объекта *Cursor*.

Изменим приоритет, начальную и конечную даты одного задания. Создадим функцию обновления данных, которая обновляет конкретное задание, находя его по идентификатору задания.

In [10]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

def update_task(conn, task):
    """
    обновляет приоритет, начальную дату, конечную дату задания
    """
    sql = ''' UPDATE task
              SET priority = ? ,
                  begin_date = ? ,
                  end_date = ?
              WHERE id = ?'''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()


def main():
    database = "projects.sqlite"

    # create a database connection
    conn = create_connection(database)
    with conn:
        update_task(conn, (2, '2022-02-21', '2022-02-25', 2))

Создадим соединение и передадим объект соединения в функцию обновления.

In [11]:
if __name__ == '__main__':
    main()

Убедимся с помощью интерфейса командной строки, что изменения произошли.

<img src="../Img/sqlite3_command_line_3.png" style="width:100%">

<a id=6></a> [К оглавлению](#Home)

### 6. Удаление данных

В этом разделе научимся с помощью Python удалять данные в созданной ранее базе данных. Удалим одно из заданий. Выполним следующие шаги алгоритма удаления данных.

1. Подключимся к базе данных SQLite, создав объект *Connection*.
2. Создадим объект *Cursor*, вызвав метод `cursor()` объекта *Connection*.
3. Выполним команду `DELETE` с помощью вызова метода `execute()` объекта *Cursor*. При необходимости передачи аргументов в метод *execute()* используем знаки вопроса (?).

Создадим функцию удаления задания, одним из параметров функции станет идентификатор удаляемого задания. Также создадим функцию удаления всех заданий.

In [12]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

def delete_task(conn, id):
    """
    Удаляет задание по идентификатору
    """
    sql = 'DELETE FROM task WHERE id=?'
    cur = conn.cursor()
    cur.execute(sql, (id,))
    conn.commit()

def delete_all_tasks(conn):
    """
    Удаляет все задания
    """
    sql = 'DELETE FROM tasks'
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

def main():
    database = "projects.sqlite"

    conn = create_connection(database)
    with conn:
        delete_task(conn, 2);
        # delete_all_tasks(conn);

Выполним шаги алгоритма по удалению задания 2.

In [13]:
if __name__ == '__main__':
    main()

Проверим результат с помощью интерфейса командной строки, запросив все записи таблицы *tasks* с помощью SQL-запроса

```
select * from tasks;
```

<img src="../Img/sqlite3_command_line_4.png" style="width:100%">

Мы убедились, что в таблице заданий осталось на одно задание меньше.

<a id=7></a> [К оглавлению](#Home)

### 7. Запросы к данным

В этом разделе научимся извлекать данные из базы данных SQLite с помощью Python.

Для выполнения запроса к базе данных с помошщью Python выполним шаги следующего алгоритма.

1. Подключимся к базе данных SQLite, создав объект *Connection*.
2. Создадим объект *Cursor*, вызвав метод `cursor()` объекта *Connection*.
3. Вполним команду SELECT.
4. Выполним метод `fetchall()` курсора для получения данных.
5. Пройдем в цикле по элементам курсора и обработаем каждую строку отдельно.

<img src="../Img/Python-SQLite-Sample-Database.jpg" style="width:50%">

Создадим функцию, которая отбирает все записи таблицы заданий и функцию, отбирающую задания по приоритету.

In [14]:
def select_all_tasks(conn):
    """
    Запрашивает все записи таблицы заданий

    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM task")

    rows = cur.fetchall()

    for row in rows:
        print(row)

def select_task_by_priority(conn, priority):
    """
    Отбирает задания по приоритету
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM task WHERE priority=?", (priority,))

    rows = cur.fetchall()

    for row in rows:
        print(row)

В функции *select_task_by_priority()* мы выбрали задачи на основе определенного приоритета. Знак вопроса (?) в запросе является символом подстановки (заполнителем). Выполняя команду SELECT, курсор заменяет символ подстановки параметром функции. Метод `fetchall()` извлекает все найденные задачи.

функция main() создаёт соединение и вызывает функцию запроса.

In [15]:
def main():
    database = "projects.sqlite"

    # create a database connection
    conn = create_connection(database)
    with conn:
        print("1. Query a task by priority:")
        select_task_by_priority(conn, 1)

        print("2. Query all the tasks")
        select_all_tasks(conn)

In [16]:
main()

1. Query a task by priority:
(1, 'Analyze the requirements of the app', 1, 1, 1, '2022-02-10', '2022-02-15')
2. Query all the tasks
(1, 'Analyze the requirements of the app', 1, 1, 1, '2022-02-10', '2022-02-15')
(3, 'Create documentation', 2, 1, 1, '2022-02-28', '2022-03-07')
